In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import pathlib
from keras.layers import  Dense, Conv2D, MaxPool2D,Flatten, Activation
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

In [ ]:
img_width, img_height = 150,150 # width & height of input image
input_depth = 3
train_data_dir      = '/content/drive/MyDrive/PROJET_Mohamed_ELGAZI/Deep_Learning/training' #data training path
validation_data_dir = '/content/drive/MyDrive/PROJET_Mohamed_ELGAZI/Deep_Learning/validation' #data testing path
epochs = 20 #number of training epoch
batch_size = 64 #training batch size

In [ ]:
# define image generator for Keras,
# here, we map pixel intensity to 0-1
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen  = ImageDataGenerator(rescale=1/255)

In [ ]:
# read image batch by batch
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    color_mode='rgb',
    target_size=(img_width,img_height),#input image size
    batch_size=batch_size,#batch size
    class_mode='categorical')
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    color_mode='rgb',
    target_size=(img_width,img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 6094 images belonging to 15 classes.
Found 1883 images belonging to 15 classes.


In [ ]:
# define number of filters and nodes in the fully connected layer
NUMB_FILTER_L1 = 20
NUMB_FILTER_L2 = 20
NUMB_FILTER_L3 = 20
NUMB_NODE_FC_LAYER = 10

#define input image order shape
if K.image_data_format() == 'channels_first':
    input_shape_val = (input_depth, img_width, img_height)
else:
    input_shape_val = (img_width, img_height, input_depth)

#define the network
model = Sequential()

# Layer 1
model.add(Conv2D(NUMB_FILTER_L1, (5, 5), 
                 input_shape=input_shape_val, 
                 padding='same', name='input_tensor'))
model.add(Activation('relu'))
model.add(MaxPool2D((2, 2)))

# Layer 2
model.add(Conv2D(NUMB_FILTER_L2, (5, 5), padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D((2, 2)))

# Layer 3
model.add(Conv2D(NUMB_FILTER_L3, (5, 5), padding='same'))
model.add(Activation('relu'))

# flattening the model for fully connected layer
model.add(Flatten())

# fully connected layer
model.add(Dense(NUMB_NODE_FC_LAYER, activation='relu'))

# output layer
model.add(Dense(train_generator.num_classes, 
                activation='softmax', name='output_tensor'))

# Compilile the network
model.compile(loss='categorical_crossentropy',
              optimizer='Adam', metrics=['accuracy'])
# Show the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_tensor (Conv2D)       (None, 150, 150, 20)      1520      
                                                                 
 activation (Activation)     (None, 150, 150, 20)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 75, 20)       0         
 )                                                               
                                                                 
 conv2d (Conv2D)             (None, 75, 75, 20)        10020     
                                                                 
 activation_1 (Activation)   (None, 75, 75, 20)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 37, 37, 20)       0         
 2D)                                                    

In [ ]:
# Train and test the network
model.fit_generator(
    train_generator,#our training generator
    #number of iteration per epoch = number of data / batch size
    steps_per_epoch=np.floor(train_generator.n/batch_size),
    epochs=epochs,#number of epoch
    validation_data=validation_generator,#our validation generator
    #number of iteration per epoch = number of data / batch size
    validation_steps=np.floor(validation_generator.n / batch_size))

In [ ]:
print("Training is done!")
model.save('/content/drive/MyDrive/PROJET_Mohamed_ELGAZI/Deep_Learning/model/model.h5')
print("Model is successfully stored!")

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_tensor (Conv2D)       (None, 150, 150, 20)      1520      
                                                                 
 activation (Activation)     (None, 150, 150, 20)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 75, 20)       0         
 )                                                               
                                                                 
 conv2d (Conv2D)             (None, 75, 75, 20)        10020     
                                                                 
 activation_1 (Activation)   (None, 75, 75, 20)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 37, 37, 20)       0         
 2D)                                                    